In [5]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [6]:
# Load Purchase Data CSV
rawdata = "purchase_data.csv"

In [7]:
# Read Purchasing File and store into Pandas data frame
rawdata_df = pd.read_csv("purchase_data.csv")
rawdata_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [9]:
#PLAYER COUNT
# Calculate the Number of Unique Players
player_demographics = rawdata_df.loc[:, ["Gender", "SN", "Age"]]
player_demographics = player_demographics.drop_duplicates()
player_count = player_demographics.count()[0]

# Display the total number of players
pd.DataFrame({"Total Players": [player_count]})

,Total Players
0,576


In [23]:
#PURCHASING ANALYSIS
# Calculate unique items, average price, etc.
item_count = len(rawdata_df["Item ID"].unique())
average_item_price = rawdata_df["Price"].mean()
total_purchase_value = rawdata_df["Price"].sum()
purchase_count = rawdata_df["Price"].count()

# Create a DataFrame to hold results
summary_table = pd.DataFrame({"Number of Unique Items": item_count,
                              "Total Sales": [total_purchase_value],
                              "Number of Purchases": [purchase_count],
                              "Average Price per Item": [average_item_price]})

# Organize/Format Data
summary_table = summary_table.round(2)
summary_table ["Average Price per Item"] = summary_table["Average Price per Item"].map("${:,.2f}".format)
summary_table ["Number of Purchases"] = summary_table["Number of Purchases"].map("{:,}".format)
summary_table ["Total Sales"] = summary_table["Total Sales"].map("${:,.2f}".format)
summary_table = summary_table.loc[:,["Number of Unique Items", "Average Price per Item", "Number of Purchases", "Total Sales"]]

# Display summary_table
display(summary_table)

,Number of Unique Items,Average Price per Item,Number of Purchases,Total Sales
0,179,$3.05,780,"$2,379.77"


In [15]:
#GENDER DEMOGRAPHICS
# Calculate the Number and Percentage of Players by Gender
gender_demographics_totals = player_demographics["Gender"].value_counts()
gender_demographics_percents = gender_demographics_totals / player_count * 100
gender_demographics = pd.DataFrame({"Count": gender_demographics_totals, "Percentage": gender_demographics_percents})

# Organize/Format Data
gender_demographics = gender_demographics.round(2)

# Display gender_demographics
display(gender_demographics)

,Count,Percentage
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [16]:
#PURCHASING ANALYSIS (GENDER)
# Calculate purchase count, avg. purchase price- by gender
gender_purchase_total = rawdata_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = rawdata_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_counts = rawdata_df.groupby(["Gender"]).count()["Price"].rename("Purchase Count")

# Calculate avg. purchase total per person etc. by gender
normalized_total = gender_purchase_total / gender_demographics["Count"]

# Create DataFrame
gender_analysis = pd.DataFrame({"Purchase Count": gender_counts, "Average Purchase Price": gender_average, "Total Purchase Value": gender_purchase_total, "Normalized Totals": normalized_total})

# Organize/Format Data
gender_analysis["Average Purchase Price"] = gender_analysis["Average Purchase Price"].map("${:,.2f}".format)
gender_analysis["Total Purchase Value"] = gender_analysis["Total Purchase Value"].map("${:,.2f}".format)
gender_analysis ["Purchase Count"] = gender_analysis["Purchase Count"].map("{:,}".format)
gender_analysis ["Avg Total per Person"] = gender_analysis["Normalized Totals"].map("${:,.2f}".format)
gender_analysis = gender_analysis.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total per Person"]]

# Display gender_analysis
display(gender_analysis)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [18]:
#AGE DEMOGRAPHICS
# Establish the bins
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Categorize the existing players using the age bins. Hint: use pd.cut()
player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=group_names)

# Calculate the numbers and percentages by age group
age_demographics_totals = player_demographics["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals / player_count * 100
age_demographics = pd.DataFrame({"Player Count": age_demographics_totals, "% of Players": age_demographics_percents})

# Format Data: Rount to two decimal places
age_demographics = age_demographics.round(2)

# Display age_demographics
age_demographics.sort_index()

,Player Count,% of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [19]:
#PURCHASING ANALYSIS (AGE)
# Bin the purchase_data data frame by age
rawdata_df["Age Ranges"] = pd.cut(rawdata_df["Age"], age_bins, labels=group_names)

# Run basic calculations to obtain purchase count, avg. purchase price,
age_purchase_total = rawdata_df.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_average = rawdata_df.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_counts = rawdata_df.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

# Run basic calculations to obtain avg. purchase total per person
normalized_total = age_purchase_total / age_demographics["Player Count"]

# Create a summary data frame to hold the results
age_analysis = pd.DataFrame({"Purchase Count": age_counts, "Average Purchase Price": age_average, "Total Purchase Value": age_purchase_total, "Normalized Totals": normalized_total})

# Organize/Format Data
age_analysis["Average Purchase Price"] = age_analysis["Average Purchase Price"].map("${:,.2f}".format)
age_analysis["Total Purchase Value"] = age_analysis["Total Purchase Value"].map("${:,.2f}".format)
age_analysis ["Purchase Count"] = age_analysis["Purchase Count"].map("{:,}".format)
age_analysis["Avg Total Purchase per Person"] = age_analysis["Normalized Totals"].map("${:,.2f}".format)
age_analysis = age_analysis.loc[:, ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]

# Display the Age Table
display(age_analysis)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [20]:
#TOP SPENDERS
# Run basic calculations to obtain the results in the table below
user_total = rawdata_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_average = rawdata_df.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = rawdata_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")

# Create a summary data frame to hold the results
user_data = pd.DataFrame({"Total Purchase Value": user_total, "Average Purchase Price": user_average, "Purchase Count": user_count})


# Sort the total purchase value column in descending order
user_sorted = user_data.sort_values("Total Purchase Value", ascending=False)

# Organize/Format Data
user_sorted["Average Purchase Price"] = user_sorted["Average Purchase Price"].map("${:,.2f}".format)
user_sorted["Total Purchase Value"] = user_sorted["Total Purchase Value"].map("${:,.2f}".format)
user_sorted = user_sorted.loc[:,["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

# Display a preview of the summary data frame
display(user_sorted)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
...,...,...,...
Ililsasya43,1,$1.02,$1.02
Irilis75,1,$1.02,$1.02
Aidai61,1,$1.01,$1.01


In [21]:
#MOST POPULAR ITEMS
# Retrieve the Item ID, Item Name, and Item Price columns
item_data = rawdata_df.loc[:,["Item ID", "Item Name", "Price"]]

# Group by Item ID and Item Name. 
#Perform calculations to obtain purchase count, average item price, and total purchase value
total_item_purchase = item_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
average_item_purchase = item_data.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_count = item_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")

# Create a summary data frame to hold the results
item_data_pd = pd.DataFrame({"Total Purchase Value": total_item_purchase, "Item Price": average_item_purchase, "Purchase Count": item_count})

# Sort the purchase count column in descending order
item_data_count_sorted = item_data_pd.sort_values("Purchase Count", ascending=False)

# Organize/Format Data
item_data_count_sorted["Item Price"] = item_data_count_sorted["Item Price"].map("${:,.2f}".format)
item_data_count_sorted["Purchase Count"] = item_data_count_sorted["Purchase Count"].map("{:,}".format)
item_data_count_sorted["Total Purchase Value"] = item_data_count_sorted["Total Purchase Value"].map("${:,.2f}".format)
item_popularity = item_data_count_sorted.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

# Display a preview of the summary data frame
display(item_popularity)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
...,...,...,...,...
42,The Decapitator,1,$1.75,$1.75
51,Endbringer,1,$4.66,$4.66
118,"Ghost Reaver, Longsword of Magic",1,$2.17,$2.17


In [22]:
#MOST PROFITABLE ITEMS
# Sort the above table by total purchase value in descending order
item_total_purchase = item_data_pd.sort_values("Total Purchase Value", ascending=False)

# Organize/Format Data
item_total_purchase["Item Price"] = item_total_purchase["Item Price"].map("${:,.2f}".format)
item_total_purchase["Purchase Count"] = item_total_purchase["Purchase Count"].map("{:,}".format)
item_total_purchase["Total Purchase Value"] = item_total_purchase["Total Purchase Value"].map("${:,.2f}".format)
item_profitable = item_total_purchase.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

# Display a preview of the data frame
display(item_profitable)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
...,...,...,...,...
28,"Flux, Destroyer of Due Diligence",2,$1.06,$2.12
125,Whistling Mithril Warblade,2,$1.00,$2.00
126,Exiled Mithril Longsword,1,$2.00,$2.00


In [ ]:
#THE END